In [5]:
import pandas as pd


In [115]:
bicycle_search = pd.read_csv('./datasets/Google_trends_data/Bicycle_search.csv')
road_search = pd.read_csv('./datasets/Google_trends_data/Roadbicycle_search.csv')
mountain_search = pd.read_csv('./datasets/Google_trends_data/Mountainbicycle_search.csv')
BMX_search = pd.read_csv('./datasets/Google_trends_data/BMXbicycle_search.csv')
hybrid_search = pd.read_csv('./datasets/Google_trends_data/Hybridbicycle_search.csv')

sales_df = pd.read_csv('./datasets/sales_df.csv')


In [116]:

print(bicycle_search.head())
print(road_search.head())
print(mountain_search.head())
print(BMX_search.head())
print(hybrid_search.head())
print(sales_df.head())

         Week  Bicycle
0  10/20/2019       44
1  10/27/2019       41
2   11/3/2019       41
3  11/10/2019       40
4  11/17/2019       41
         Week  Road Bicycle
0  10/20/2019            44
1  10/27/2019            36
2   11/3/2019            44
3  11/10/2019            40
4  11/17/2019            39
         Week  Mountain Bicycle
0  10/20/2019                29
1  10/27/2019                24
2   11/3/2019                26
3  11/10/2019                26
4  11/17/2019                27
         Week  BMX
0  10/20/2019   28
1  10/27/2019   26
2   11/3/2019   28
3  11/10/2019   25
4  11/17/2019   27
         Week  Hybrid bicycle
0  10/20/2019              20
1  10/27/2019              17
2   11/3/2019              21
3  11/10/2019              20
4  11/17/2019              20
           bike_name    bike_type date_ordered  quantity
0  UrbanCommuter 500  Hybrid Bike   2022-05-14         1
1  BMX Freestyle Pro          BMX   2024-03-06         2
2     Roadmaster Pro    Road Bike   2

In [117]:
print(bicycle_search.shape)
print(road_search.shape)
print(mountain_search.shape)
print(BMX_search.shape)
print(hybrid_search.shape)

(262, 2)
(262, 2)
(262, 2)
(262, 2)
(262, 2)


In [118]:
# Check the number of null values in each column
null_counts = BMX_search.isnull().sum()

# Display the number of null values
print(null_counts)


Week    0
BMX     0
dtype: int64


In [119]:
bicycle_search.columns = ['date', 'search_value']
road_search.columns = ['date', 'search_value']
mountain_search.columns = ['date', 'search_value']
BMX_search.columns = ['date', 'search_value']
hybrid_search.columns = ['date', 'search_value']

bicycle_search['date'] = pd.to_datetime(bicycle_search['date'])
road_search['date'] = pd.to_datetime(road_search['date'])
mountain_search['date'] = pd.to_datetime(mountain_search['date'])
BMX_search['date'] = pd.to_datetime(BMX_search['date'])
hybrid_search['date'] = pd.to_datetime(hybrid_search['date'])

sales_df['date_ordered'] = pd.to_datetime(sales_df['date_ordered'])


In [120]:
#add the 6 remaining days of the week in the dataframes
def populate_df_week(df):
    new_rows=[]
    for index, row in bicycle_search.iterrows():
        for i in range (0,7):
            new_row = row.copy()
            # print(type(row['date']))
            # print(type(row['search_value']))
            new_rows.append([row['date'] - pd.Timedelta(days=i), row['search_value'],])  
    return pd.DataFrame(new_rows, columns=df.columns)

In [121]:

bicycle_search = populate_df_week(bicycle_search)
road_search = populate_df_week(road_search)
mountain_search = populate_df_week(mountain_search)
hybrid_search = populate_df_week(hybrid_search)
BMX_search = populate_df_week(BMX_search)

In [122]:
road_search.columns = ['date', 'bike_type_search_value']
mountain_search.columns = ['date', 'bike_type_search_value']
BMX_search.columns = ['date', 'bike_type_search_value']
hybrid_search.columns = ['date', 'bike_type_search_value']

In [123]:

sales_df = sales_df.merge(bicycle_search[['date', 'search_value']], 
                           left_on='date_ordered', 
                           right_on='date', 
                           how='inner')

# Display the updated sales_df
print(sales_df.head())
# ... existing code ...

           bike_name      bike_type date_ordered  quantity       date  \
0  UrbanCommuter 500    Hybrid Bike   2022-05-14         1 2022-05-14   
1      AirMaster 300            BMX   2022-05-14         1 2022-05-14   
2  UrbanCommuter 500    Hybrid Bike   2022-05-14         2 2022-05-14   
3     TrailBlazer XT  Mountain Bike   2022-05-14         3 2022-05-14   
4  BMX Freestyle Pro            BMX   2022-05-14         3 2022-05-14   

   search_value  
0            61  
1            61  
2            61  
3            61  
4            61  


In [126]:
# ... existing code ...

# Function to merge with the appropriate search dataframe based on bike_type
def merge_with_bike_type(row):
    if row['bike_type'] == 'Road Bike':
        return road_search
    elif row['bike_type'] == 'Mountain Bike':
        return mountain_search
    elif row['bike_type'] == 'BMX':
        return BMX_search
    elif row['bike_type'] == 'Hybrid Bike':
        return hybrid_search
    else:
        return None  # or handle as needed

In [127]:
# Create a new DataFrame to hold the merged results
merged_results = []

# Iterate through each row in sales_df
for index, row in sales_df.iterrows():
    search_df = merge_with_bike_type(row)
    # Convert the row to a DataFrame for merging
    row_df = pd.DataFrame([row])  # Convert Series to DataFrame
    # Merge with the appropriate search dataframe
    merged_row = row_df.merge(search_df[['date', 'bike_type_search_value']], 
                                left_on='date_ordered', 
                                right_on='date', 
                                how='inner')
    merged_results.append(merged_row)

    
# Convert the list of merged results back to a DataFrame
merged_sales_df = pd.concat(merged_results, ignore_index=True)

# Display the updated merged_sales_df
print(merged_sales_df.head())


           bike_name    bike_type date_ordered  quantity     date_x  \
0  UrbanCommuter 500  Hybrid Bike   2022-05-14         1 2022-05-14   
1  UrbanCommuter 500  Hybrid Bike   2022-05-14         1 2022-05-14   
2  UrbanCommuter 500  Hybrid Bike   2022-05-14         1 2022-05-14   
3  UrbanCommuter 500  Hybrid Bike   2022-05-14         1 2022-05-14   
4  UrbanCommuter 500  Hybrid Bike   2022-05-14         1 2022-05-14   

   search_value     date_y  bike_type_search_value  
0            61 2022-05-14                      61  
1            61 2022-05-14                      61  
2            61 2022-05-14                      64  
3            61 2022-05-14                      64  
4            61 2022-05-14                      64  


In [137]:
# Sanity check
print(merged_sales_df.shape)

difference_count2 = (merged_sales_df['date_x'] != merged_sales_df['date_y'] != merged_sales_df['date_ordered']).sum()
print(difference_count2)


difference_count = (merged_sales_df['search_value'] != merged_sales_df['bike_type_search_value']).sum()
print(f"The columns 'search_value' and 'bike_type_search_value' are different {difference_count} times.")

(140000, 7)
0
The columns 'search_value' and 'bike_type_search_value' are different 52905 times.


In [138]:
merged_sales_df = merged_sales_df.drop(columns=['date_x'])
merged_sales_df = merged_sales_df.drop(columns=['date_y'])

           bike_name    bike_type date_ordered  quantity  search_value  \
0  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
1  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
2  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
3  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
4  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   

   bike_type_search_value  
0                      61  
1                      61  
2                      64  
3                      64  
4                      64  


In [139]:
print(merged_sales_df.head())


           bike_name    bike_type date_ordered  quantity  search_value  \
0  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
1  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
2  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
3  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   
4  UrbanCommuter 500  Hybrid Bike   2022-05-14         1            61   

   bike_type_search_value  
0                      61  
1                      61  
2                      64  
3                      64  
4                      64  


In [140]:
merged_sales_df.to_csv('./datasets/sales_and_trends_df.csv', index=False)